In [1]:
import os
import zipfile
import shutil
from docx import Document
from transformers import pipeline

# Extract Tender Documents

In [ ]:
from ExtractTenders import extract

search_path = r"C:\Users\Mitch\Tenders"
copy_path = r"C:\Users\Mitch\SpecDocs"
extract(search_path, copy_path)

# Extract Text

In [2]:
from ReadFiles import extract_doc_by_header

directory_path = r"C:/Users/Mitch/SpecDocs"
header = "Background"
docs = []

for doc in os.listdir(directory_path):
    doc_path = os.path.join(directory_path, doc)
    try:
        docs.append(extract_doc_by_header(doc_path, header))
    except:
        continue

# remove those docs without background
docs = [doc for doc in docs if doc != ""]

# Load Summarisation Pipeline

In [7]:
class Summariser:
    def __init__(self, max_length, min_length, do_sample=False):
        """ Summarises text inputs """
        # bart-large-xsum has default vocab 50,265
        # each layers has default 1024
        self.summarizer = pipeline('summarization', model='facebook/bart-large-xsum')
        self.max_length = max_length
        self.min_length = min_length
        self.do_sample = do_sample
        self.summarised_docs = [] # need to change this to dict

    def summarise_docs(self, docs):
        """ 
        max_length (int): Maximum length of the generated summary.
        min_length (int): Minimum length of the generated summary.
        do_sample (bool): Whether to use greedy sampling when generating summaries.
        
        Returns: list of summarised docs
        """
        
        for doc in docs:
            try: # some documents still too long
                summary = self.summarizer(doc, max_length=self.max_length, min_length=self.min_length, do_sample=self.do_sample)
                self.summarised_docs.append(summary[0]['summary_text'])
            except:
                continue

        return self.summarised_docs
    
    def doc_summary(self):
        
        # need to change into dictionary to return individual summary by reference number
        return self.summarised_docs # summarised_docs[ref]

# Summarise Documents

In [8]:
summariser = Summariser(max_length=50, min_length=30, do_sample=False)

summarised_docs = summariser.summarise_docs(docs[1:5])

for i, summary in enumerate(summarised_docs):
    print(f"Summary for article {i}:\n{summary}")

Summary for article 0:
The Art Gallery of Western Australia (AGWA, the Customer) is looking for a consultant to assist with the day to day running of its Rooftop Bar and AGWA Café and Bar.
Summary for article 1:
The Western Australian government is seeking a new director for the Child and Adolescent Health Service - Community Health (CAHS-CH) in Western Australia.
Summary for article 2:
Belfast City Council is looking for a new blood transfusion system for children and young people in the north west of the city and the surrounding area.
Summary for article 3:
A contract has been submitted for the supply of a surgical microscope to the Child and Adolescent Health Service (CAHS) in Londonderry.
